## Project - Nobel Prize Winners and University Data

In this project, data from two different sources is combined to explore connections between Nobel Prize winners and universities. Nobel laureate information is imported from a Kaggle CSV file into a SQLite database. University data is retrieved from DBpedia using a SPARQL query, which is also stored in the same database.

To link the datasets, a SQL view is created based on matching university names from both tables. This makes it possible to analyze which universities are associated with Nobel laureates, along with additional information like the number of students and Wikipedia page IDs of the universities.

The key column used for linking the two datasets is organizationName in the Nobel laureates data. This field represents the university or research institution where the laureate was affiliated at the time of receiving the Nobel Prize. It is matched against the university names retrieved from DBpedia to establish a connection. For example, entries like Harvard University or Szeged University are used to join the tables and perform further analysis.

If this notebook does not render or execute correctly in your environment, it is also available on my GitHub repository: https://github.com/Fab2102/dke_project/blob/main/project.ipynb

<br>

## Installing and importing all dependencies
- SPARQL Wrapper
    - to query SPARQL endpoints like DBpedia

- sqlite3
    - to work with local SQL databases

- pandas
    - to handle and analyze CSV data

In [ ]:
%pip install SPARQLWrapper
%pip install pandas

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import sqlite3 
import pandas as pd

<br>

## Load SQL Extension and create/connect to SQLite Database

- Enabling SQL magic commands in jupyter

- Creating an SQL Lite database named "data.db"

- Creating a python connection object for programmatic access

In [ ]:
%load_ext sql
%sql sqlite:///data.db

In [ ]:
conn = sqlite3.connect('data.db')

<br><br><br>

# CSV data
The dataset `"Nobel Prize Winners: 1901 to 2023"` is loaded in with pandas and then turned into an SQL table. This makes it easier to join with RDF data from a SPARQL query later on. The dataset was downloaded from Kaggle and can be found [here](https://www.kaggle.com/datasets/sazidthe1/nobel-prize-data).

<br>

## Processing CSV

- Keep only the following 4 columns: `year`, `category`, `fullName`, `organizationName` 

- Transforming the pandas DataFrame into an SQL Table named "nobel"

- Printing out the pandas DataFrame

In [ ]:
df_csv = pd.read_csv("nobel_laureates_data.csv", encoding='utf8').iloc[:, [0, 1, 5, 13]].copy()
df_csv.to_sql("nobel", conn, if_exists="replace", index=False)
df_csv

<br>

## Showing CSV data via SQL query

The following SQL query returns the first 10 rows from the `nobel` table.

In [ ]:
%sql select * from nobel limit 10;

<br><br><br>

# RDF data

In this section, RDF data is retrieved from the DBpedia SPARQL endpoint. The query fetches information about universities, including their English names, the number of students and their Wikipedia page IDs.

<br>

## Reading in DBpedia data via SPARQL

- Connects to the DBpedia SPARQL endpoint by using `SPARQLWrapper`

- Queries following information: `universityName`, `numStudents` and `wikiPageID`

- Returns results in a JSON format and prints out the first 10 entries

In [ ]:
sparql = SPARQLWrapper("https://dbpedia.org/sparql")

sparql.setQuery("""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbo:  <http://dbpedia.org/ontology/>

SELECT DISTINCT
  ?universityName
  ?numStudents
  ?wikiPageID
WHERE {
  ?university rdf:type dbo:University .
  ?university rdfs:label ?universityName .
  ?university dbo:numberOfStudents ?numStudents .
  ?university dbo:wikiPageID ?wikiPageID .
  FILTER(lang(?universityName) = "en")
}
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()["results"]["bindings"]

for res in results[:10]:
    name = res["universityName"]["value"]
    num_students = res["numStudents"]["value"]
    wiki_id = res["wikiPageID"]["value"]
    print(f"{name}, {num_students}, {wiki_id}")


<br>

## Transforming RDF data to CSV/SQL

- Uses a list comprehension to convert the SPARQL results into a pandas DataFrame

- Saves the DataFrame as a CSV file called `universites.csv`

- Transforms the CSV into an SQL Table called `universities`

- Printing out the pandas DataFrame


In [ ]:
df_rdf = pd.DataFrame([{
    "UniversityName": res["universityName"]["value"],
    "NumStudents": res.get("numStudents", {}).get("value", ""),
    "WikiPageID": res.get("wikiPageID", {}).get("value", "")
} for res in results])

df_rdf.to_csv("universites.csv", sep=";", index=False, encoding="utf8")
pd.read_csv("universites.csv", delimiter=";").to_sql("universities", conn, if_exists="replace", index=False);

df_rdf


<br>

## Showing RDF data via SQL query
Displays the first 10 rows from the `universities` table to preview the imported RDF data

In [ ]:
%sql select * from universities limit 10;

<br><br><br>

# Combining CSV + RDF

- Drops the view `nobel_universities` if it already exists for safety reasons

- Creates a new SQL view by joining Nobel Prize winners (CSV) with university data (RDF) on matching University Names

- Displays the first 10 rows from the joined view for a quick preview

In [ ]:
%%sql

DROP VIEW IF EXISTS nobel_universities;

CREATE VIEW nobel_universities AS
SELECT a.year, a.category, a.fullName, b.UniversityName, b.NumStudents, b.WikiPageID
FROM nobel a
JOIN universities b ON a.organizationName = b.UniversityName;

SELECT * FROM nobel_universities LIMIT 10;


# 5 complex queries

### 1. Rank universities by total laureates

In [ ]:
%%sql

SELECT 
  UniversityName, 
  COUNT(*) AS laureate_count, 
  RANK() OVER (ORDER BY COUNT(*) DESC) AS rank
FROM nobel_universities
GROUP BY UniversityName
ORDER BY laureate_count DESC
LIMIT 10;


### 2. Laureate density per university

In [ ]:
%%sql

SELECT 
  UniversityName, 
  COUNT(*) AS laureate_count, 
  NumStudents,
  ROUND(CAST(COUNT(*) AS REAL)/NumStudents, 6) AS laureates_per_student
FROM nobel_universities
GROUP BY UniversityName, NumStudents
HAVING NumStudents > 0
ORDER BY laureates_per_student DESC
LIMIT 10;


### 3. Nobel Laureates whose names start with 'C'

In [ ]:
%%sql

SELECT 
  fullName, 
  category, 
  UniversityName, 
  NumStudents, 
  WikiPageID
FROM nobel_universities
WHERE UPPER(fullName) LIKE 'C%'
LIMIT 10;


### 4. Universities Ranked by the Number of Distinct Nobel Prize Categories

In [ ]:
%%sql

SELECT 
  UniversityName,
  COUNT(*) AS total_laureates,
  COUNT(DISTINCT category) AS distinct_categories,
  GROUP_CONCAT(DISTINCT category) AS categories
FROM nobel_universities
GROUP BY UniversityName
HAVING COUNT(DISTINCT category) > 1
ORDER BY distinct_categories DESC, total_laureates DESC
LIMIT 10;


### 5. Top Universities by Nobel Laureate Count per Category

In [ ]:
%%sql

SELECT
  UniversityName,
  category,
  PrizeCount
FROM (
  SELECT
    UniversityName,
    category,
    COUNT(*) AS PrizeCount,
    RANK() OVER (PARTITION BY category ORDER BY COUNT(*) DESC) AS rk
  FROM nobel_universities
  GROUP BY UniversityName, category
) t
WHERE rk = 1;
